In [1]:
from quaternions import *
from networks import *
from helpers_train_test import *
from liegroups.numpy import SO3
import torch
from convex_layers import *

In [2]:
path = 'saved_data/kitti/kitti_model_A_sym_seq_00_12-31-2019-19-25-55.pt'
checkpoint = torch.load(path)
args = checkpoint['args']
print(args)
device = torch.device('cuda:0') if args.cuda else torch.device('cpu')
tensor_type = torch.double if args.double else torch.float

transform = None
seqs_base_path = '/media/m2-drive/datasets/KITTI/single_files'
if args.megalith:
    seqs_base_path = '/media/datasets/KITTI/single_files'

kitti_data_pickle_file = 'kitti/kitti_singlefile_data_sequence_{}_delta_2_reverse_True_min_turn_1.0.pickle'.format(args.seq)
valid_loader = DataLoader(KITTIVODatasetPreTransformed(kitti_data_pickle_file, use_flow=args.optical_flow, seqs_base_path=seqs_base_path, transform_img=transform, run_type='test', seq_prefix=seq_prefix),
                            batch_size=args.batch_size_test, pin_memory=False,
                            shuffle=True, num_workers=args.num_workers, drop_last=True)

kitti_data_pickle_file_new = 'kitti/kitti_singlefile_data_sequence_01_delta_2_reverse_True_min_turn_1.0.pickle'
valid_loader_2 = DataLoader(KITTIVODatasetPreTransformed(kitti_data_pickle_file_new, use_flow=args.optical_flow, seqs_base_path=seqs_base_path, transform_img=transform, run_type='test', seq_prefix=seq_prefix),
                            batch_size=args.batch_size_test, pin_memory=False,
                            shuffle=True, num_workers=args.num_workers, drop_last=True)
#Train and test with new representation
dim_in = 2 if args.optical_flow else 6

FileNotFoundError: [Errno 2] No such file or directory: 'saved_data/kitti/kitti_model_A_sym_seq_00_12-31-2019-19-25-55.pt'

In [3]:
print('==============Using A (Sym) MODEL====================')
model = QuatFlowNet(enforce_psd=False, unit_frob_norm=args.unit_frob, dim_in=dim_in, batchnorm=args.batchnorm).to(device=device, dtype=tensor_type)
valid_loader.dataset.rotmat_targets = False
valid_loader2.dataset.rotmat_targets = False

print('Loading data...')
model.load_state_dict(checkpoint['model'], strict=False)
print('Done')

==============Using A (Sym) MODEL====================


NameError: name 'args' is not defined

In [ ]:
def sorted_els(A):
    el, _ = np.linalg.eig(A)
    el.sort(axis=1)
    return el

q_est = []
A_pred = []
q_est2 = []
A_pred2 = []

with torch.no_grad():
    for _, (x, target) in enumerate(valid_loader):
        #Move all data to appropriate device
        x = x.to(device=device, dtype=tensor_type)
        q_est.append(model.forward(x).squeeze().cpu())            
        A_pred.append(model.output_A(x).cpu())
        
    for _, (x, target) in enumerate(valid_loader2):
        #Move all data to appropriate device
        x = x.to(device=device, dtype=tensor_type)
        q_est2.append(model.forward(x).squeeze().cpu())            
        A_pred2.append(model.output_A(x).cpu())
        
A_pred = torch.cat(A_pred, dim=0)
q_est = torch.cat(q_est, dim=0)
A_pred2 = torch.cat(A_pred2, dim=0)
q_est2 = torch.cat(q_est2, dim=0)

fig, ax = plt.subplots(1, 1, sharex='col', sharey='row')
_scatter(ax, sorted_els(A_pred.numpy())[:,0], sorted_els(A_pred.numpy())[:,1], 'Seq 00', color='tab:blue')
_scatter(ax, sorted_els(A_pred2.numpy())[:,0], sorted_els(A_pred2.numpy())[:,1], 'Seq 01', color='tab:red')
ax.set_title('First/Second Eigenvalues')
ax.grid(True, which='both', color='tab:grey', linestyle='--', alpha=0.5, linewidth=0.5)
ax.set_ylabel('Second eigenvalue')
ax.set_xlabel('First eigenvalue')